In [4]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [7]:
actions = ['Hello', 'Thank you', 'Tell me', 'cat', 'I', 'cold', 'and', 'headache']

data = np.concatenate([
    np.load('dataset/seq_Hello_1708280236.npy'),
    np.load('dataset/seq_Thank you_1708280236.npy'),
    np.load('dataset/seq_I have a headache and cold_1708280236.npy')
], axis=0)

data.shape

(2332, 30, 100)

In [ ]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

In [9]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(2332, 3)

In [10]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(2098, 30, 99) (2098, 3)
(234, 30, 99) (234, 3)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                41984     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 44163 (172.51 KB)
Trainable params: 44163 (172.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/200
64/66 [============================>.] - ETA: 0s - loss: 10.4403 - acc: 0.7065
Epoch 1: val_acc improved from -inf to 0.66239, saving model to models/model.h5
66/66 [==============================] - 4s 26ms/step - loss: 10.4078 - acc: 0.7059 - val_loss: 9.5476 - val_acc: 0.6624 - lr: 0.0010
Epoch 2/200
 7/66 [==>...........................] - ETA: 1s - loss: 8.8729 - acc: 0.6429

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


64/66 [============================>.] - ETA: 0s - loss: 6.4938 - acc: 0.6538
Epoch 2: val_acc improved from 0.66239 to 0.69658, saving model to models/model.h5
66/66 [==============================] - 1s 20ms/step - loss: 6.4083 - acc: 0.6544 - val_loss: 6.1413 - val_acc: 0.6966 - lr: 0.0010
Epoch 3/200
65/66 [============================>.] - ETA: 0s - loss: 6.3944 - acc: 0.7437
Epoch 3: val_acc improved from 0.69658 to 0.80769, saving model to models/model.h5
66/66 [==============================] - 1s 19ms/step - loss: 6.3650 - acc: 0.7440 - val_loss: 5.4075 - val_acc: 0.8077 - lr: 0.0010
Epoch 4/200
66/66 [==============================] - ETA: 0s - loss: 2.1481 - acc: 0.8441
Epoch 4: val_acc improved from 0.80769 to 0.89744, saving model to models/model.h5
66/66 [==============================] - 2s 24ms/step - loss: 2.1481 - acc: 0.8441 - val_loss: 1.0041 - val_acc: 0.8974 - lr: 0.0010
Epoch 5/200
65/66 [============================>.] - ETA: 0s - loss: 4.6848 - acc: 0.7389
Epoc